# OUTIL - TROUVER SA CONVENTION COLLECTIVE

**Définition** : 

Ce notebook permet de générer les KPIs autours de l'outil Trouver sa convention collective :

 * Convertion : Taux des visites où l'utilisateur a obtenu sa convention collective


## Récupération des données

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Interval

Renseignez la date de début (incluse) et de fin (non incluse) souhaitées pour la récupération des données

In [ ]:
date_debut = '2024-10-01'
date_fin = '2024-11-01'

### Convention collectives supportées

Utilisé pour filtrer dans la requête ES et allégé le processus

In [ ]:
cc_ids = ['2511','1596','1597','3127','1404','3043','1517','1527','2941','1043','2609','2596','2614','1483','3239','2264','2216','2148','2098','2120','1996','1979','1702','1672','1516','1518','1606','1501','1505','1486','1480','1351','1266','1147','1090','0843','0675','0292','0275','0176','0086','0044','0016','0029','3248','2420','0413','0787','0573']  # Liste des identifiants des conventions collectives

### Requête ES

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "bool": {
            "should": [
              {
                "prefix": {
                  "url": "https://code.travail.gouv.fr/outils/convention-collective"
                }
              },
              {
                "prefix": {
                  "url": "https://code.travail.gouv.fr/convention-collective/3239"
                }
              }
            ]
          }
        },
        {
          "range": {
            "logfile": {
              "gte": date_debut,
              "lt": date_fin
            }
          }
        }
      ]
    }
  }
}

In [ ]:
inital_logs = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
tmp_log = inital_logs.copy()
# Nettoyer les URLs pour retirer tout ce qui suit "?"
inital_logs['url'] = inital_logs['url'].apply(lambda x: x.split('?')[0].split('#')[0])

## Taux des visites où l'utilisateur a obtenu sa convention collective

### Nombre de visites sur l'outil

In [ ]:
logs = inital_logs.copy()

total_unique_visits = len(logs['idVisit'].unique())

### Nombre de visites sur l'étape "Je ne connais pas"

In [ ]:
logs_agreements = inital_logs[
    inital_logs['url'] == 'https://code.travail.gouv.fr/outils/convention-collective/convention'
]

total_unique_visits_agreements = len(logs_agreements['idVisit'].unique())

### Nombre de visites sur l'étape "Je ne connais pas" provenant de l'outils "Trouver sa CC"

In [ ]:
logs_agreements_from_tool = inital_logs[
    inital_logs['type'] == 'visit_content'
]

# Convertir timestamp en datetime pour trier facilement
logs_agreements_from_tool['date'] = pd.to_datetime(logs_agreements_from_tool['timestamp'], unit='s')

# Trier par idVisit et timestamp
sorted_logs_agreements_from_tool = logs_agreements_from_tool.sort_values(by=['idVisit', 'date'])

def filter_visits(group):
    urls = group['url'].tolist()
    # Parcourir la liste des URLs
    for i in range(len(urls) - 1):
        if urls[i] == "https://code.travail.gouv.fr/outils/convention-collective":
            if urls[i + 1] == "https://code.travail.gouv.fr/outils/convention-collective/convention":
                return True
    return False

# Appliquer le filtre sur les groupes
grouped_sorted_logs_agreements_from_tool = sorted_logs_agreements_from_tool.groupby('idVisit')
visits_agreements_from_tool = grouped_sorted_logs_agreements_from_tool.filter(filter_visits)

# Trouver les valeurs uniques de la colonne 'idVisit'
total_visits_agreements_from_tool = len(visits_agreements_from_tool['idVisit'].unique())

### Nombre de visites ayant effectué une recherche de CC

In [ ]:
logs_agreement_search = inital_logs[
    inital_logs['type'] == 'cc_search'
]

total_agreement_search = len(logs_agreement_search['idVisit'].unique())

### Nombre de visites ayant cliqué sur une convention collective

In [ ]:
logs_agreements_select = inital_logs[
    inital_logs['type'] == 'cc_select_p1'
]

total_unique_visits_agreements_select = len(logs_agreements_select['idVisit'].unique())

### Nombre de visites passant par la recherche d'entreprise

In [ ]:
logs_enterprises = inital_logs[
    inital_logs['url'] == 'https://code.travail.gouv.fr/outils/convention-collective/entreprise'
]

total_unique_visits_enterprises = len(logs_enterprises['idVisit'].unique())

### Nombre de visites passant par la recherche d'entreprise provenant de l'outils "Trouver sa CC"

In [ ]:
logs_enterprises_from_tool = inital_logs[
    inital_logs['type'] == 'visit_content'
]

# Convertir timestamp en datetime pour trier facilement
logs_enterprises_from_tool['date'] = pd.to_datetime(logs_enterprises_from_tool['timestamp'], unit='s')

# Trier par idVisit et timestamp
sorted_logs_enterprises_from_tool = logs_enterprises_from_tool.sort_values(by=['idVisit', 'date'])

def filter_visits(group):
    urls = group['url'].tolist()
    # Parcourir la liste des URLs
    for i in range(len(urls) - 1):
        if urls[i] == "https://code.travail.gouv.fr/outils/convention-collective":
            if urls[i + 1] == "https://code.travail.gouv.fr/outils/convention-collective/entreprise":
                return True
    return False

# Appliquer le filtre sur les groupes
grouped_sorted_logs_enterprises_from_tool = sorted_logs_enterprises_from_tool.groupby('idVisit')
visits_enterprises_from_tool = grouped_sorted_logs_enterprises_from_tool.filter(filter_visits)

# Trouver les valeurs uniques de la colonne 'idVisit'
total_visits_enterprises_from_tool = len(visits_enterprises_from_tool['idVisit'].unique())

### Nombre de visite avec une recherche d'entreprise

In [ ]:
logs_enterprise_search = inital_logs[
    inital_logs['type'] == 'enterprise_search'
]

total_enterprise_search = len(logs_enterprise_search['idVisit'].unique())

### Nombre de visite avec une sélection d'entreprise

In [ ]:
logs_enterprise_select = inital_logs[
    inital_logs['type'] == 'enterprise_select'
]

total_enterprise_select = len(logs_enterprise_select['idVisit'].unique())

In [ ]:
logs_enterprise_cc_select = inital_logs[
    inital_logs['type'] == 'cc_select_p2'
]

total_enterprise_cc_select = len(logs_enterprise_cc_select['idVisit'].unique())

### Nombre de visites arrivant sur la CC 3239 à partir de l'écran de recherche par entreprise

Il existe un event du type : cc_search_type_of_users, avec l'action : click_je_n_ai_pas_d_entreprise

Cependant l'action n'est pas disponible. Il faut donc trouver une autre méthode pour le trouver en attendant que l'action soit disponible directement.

Dans le cas présent, on va regarder pour chaque visite sur la page de recherche entreprise, s'il y a la visite de la page de la convention collective 3239 juste après. Dans ce cas, on va déduire que l'utilisateur a utilisé ce lien.

In [ ]:
no_enterprise_logs = inital_logs[
    inital_logs['type'] == 'visit_content'
]

# Convertir timestamp en datetime pour trier facilement
no_enterprise_logs['date'] = pd.to_datetime(no_enterprise_logs['timestamp'], unit='s')

# Trier par idVisit et timestamp
sorted_no_enterprise_logs = no_enterprise_logs.sort_values(by=['idVisit', 'date'])

def filter_visits(group):
    urls = group['url'].tolist()
    # Parcourir la liste des URLs
    for i in range(len(urls) - 1):
        # Vérifier que l'URL actuelle est une recherche par entreprise
        if urls[i].startswith("https://code.travail.gouv.fr/outils/convention-collective/entreprise"):
            # Vérifier que l'URL suivante est la convention collective 3239
            if urls[i + 1].startswith("https://code.travail.gouv.fr/convention-collective/3239"):
                return True
    return False

# Appliquer le filtre sur les groupes
grouped_no_enterprise_logs = sorted_no_enterprise_logs.groupby('idVisit')
valid_visits = grouped_no_enterprise_logs.filter(filter_visits)

# Trouver les valeurs uniques de la colonne 'idVisit'
total_enterprise_3239 = len(valid_visits['idVisit'].unique())

### Création du tableau contenant les données de convertion

In [ ]:
# Méthodes de formatage
def format_numbers(value):
    """Formater les nombres bruts (première ligne)."""
    if isinstance(value, float):
        return f"{int(value):,}"  # Nombre sans décimales
    return f"{value:,}"

def format_percentages(value):
    """Formater les pourcentages (deuxième ligne)."""
    return f"{value:.2f}%" if isinstance(value, float) else value

def display_result(data):
    df = pd.DataFrame(data)

    # Appliquer les styles
    styled_df = (
        df.style
        .format(
            {col: format_numbers for col in df.columns}, subset=pd.IndexSlice[0, :]  # Première ligne
        )
        .format(
            {col: format_percentages for col in df.columns}, subset=pd.IndexSlice[1, :]  # Deuxième ligne
        )
    )

    return styled_df

### Taux de convertion de la 1er page "Trouver sa CC"

In [ ]:
data = {
    "Total visites": [total_unique_visits, 100],
    "P1 : Conversion": [total_visits_agreements_from_tool, (total_visits_agreements_from_tool / total_unique_visits * 100)],
    "P2 : Conversion": [total_visits_enterprises_from_tool, (total_visits_enterprises_from_tool / total_unique_visits * 100)],
}
display_result(data)

### Funnel du parcours "Je connais ma CC" 
Les visites correspondent au total des visites sur la page /outils/convention-collective/convention (qu'elles proviennent de l'outil ou non)

In [ ]:
data = {
    "P1 : Visites": [total_unique_visits_agreements, 100],
    "P1 : Recherches": [total_agreement_search, (total_agreement_search / total_unique_visits_agreements * 100)],
    "P1 : Click CC": [total_unique_visits_agreements_select, (total_unique_visits_agreements_select / total_agreement_search * 100)],
}
display_result(data)

### Funnel du parcours "Je ne connais pas ma CC"

Les visites correspondent au total des visites sur la page /outils/convention-collective/entreprise (qu'elles proviennent de l'outil ou non)


In [ ]:
data = {
    "P2 : Visites": [total_unique_visits_enterprises, 100],
    "P2 : Recherches": [total_enterprise_search, (total_enterprise_search / total_unique_visits_enterprises * 100)],
    "P2 : Sélections entreprise": [total_enterprise_select, (total_enterprise_select / total_enterprise_search * 100)],
    "P2 : Click CC": [total_enterprise_cc_select, (total_enterprise_cc_select / total_enterprise_select * 100)],
    "P2 : 3239": [total_enterprise_3239, (total_enterprise_3239 / total_unique_visits_enterprises * 100)],
}
display_result(data)

### Sauvegarde du tableau dans un fichier CSV

Le fichier est disponible dans le dossier `explorations/outputs`

In [ ]:
df.to_csv(f"./outputs/ratio_completion_trouver_sa_convention_collective_{date_debut}_{date_fin}.csv", index=False, encoding='utf-8')